## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Lincoln,AU,-34.73,135.87,59.94,75,67,21.70,broken clouds
1,1,Rongcheng,CN,30.63,117.85,58.05,90,66,3.58,broken clouds
2,2,Kisesa,TZ,-3.08,34.15,71.06,60,66,6.91,broken clouds
3,3,Halifax,CA,44.65,-63.57,39.99,78,20,1.01,few clouds
4,4,Santa Cruz,BO,-17.80,-63.17,87.84,48,4,13.65,clear sky


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature(F) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature(F) you would like for your trip? "))

What is the minimum temperature(F) you would like for your trip? 70
What is the maximum temperature(F) you would like for your trip? 85


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()

In [20]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                264
City                   264
Country                264
Lat                    264
Lng                    264
Max Temp               264
Humidity               264
Cloudiness             264
Wind Speed             264
Current Description    264
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# N/A

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kisesa,TZ,71.06,broken clouds,-3.08,34.15,
13,Bousse,BF,77.68,clear sky,12.66,-1.89,
15,Kirakira,SB,84.90,light rain,-10.45,161.92,
16,Arraial Do Cabo,BR,75.52,clear sky,-22.97,-42.03,
17,Palu,ID,79.56,light rain,-0.89,119.87,
18,Atuona,PF,77.95,broken clouds,-9.80,-139.03,
23,Victoria,HK,75.99,overcast clouds,22.29,114.16,
26,Vaini,TO,73.26,clear sky,-21.20,-175.20,
29,Coahuayana,MX,83.68,light rain,18.73,-103.68,
31,Rikitea,PF,74.93,broken clouds,-23.12,-134.97,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [43]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [46]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Bousse,BF,77.68,clear sky,12.66,-1.89,Maison d'hôtes EBEN-EZER DE BOUSSE
15,Kirakira,SB,84.90,light rain,-10.45,161.92,SanBiz Lodge
16,Arraial Do Cabo,BR,75.52,clear sky,-22.97,-42.03,Pousada Porto Praia
17,Palu,ID,79.56,light rain,-0.89,119.87,Hotel Santika Palu
18,Atuona,PF,77.95,broken clouds,-9.80,-139.03,Villa Enata


In [47]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
